In [1]:
import pandas as pd
from nltk import word_tokenize
# Remoção de acentos
from unicodedata import normalize
import _thread
import time
import requests
import re

In [2]:
MAX = 200
PARTS = 4

dados = []

BASE_URL = "http://www.barbacena.ifsudestemg.edu.br"

def get_sites(num_thread, offset):
    try:
        global MAX, PARTS, BASE_URL, dados

        for i in range(int(offset), int(offset+MAX/PARTS)):
            html = requests.get(BASE_URL+'/nodequeue/3?page='+str(i)).text

            sub_urls = re.findall(r'<a href="(/destaques.+)"', html)
            for sub_url in sub_urls:
                html = requests.get(BASE_URL+sub_url).text
                link = BASE_URL+sub_url
                data = re.findall(r'<div class="submitted">Postado em (.+),', html)[0]
                title = re.findall(r'<h1 class="title">(.+)</h1>', html)[0]
                desc = ""
                for match in re.finditer('<p class="rtejustify">(.*?)</p>', html, re.S):
                    desc += match.group(1)
                dados.append((data, title, link, desc))
    except Exception:
        import traceback
        print(traceback.format_exc())

try:
    for i in range(PARTS):
        _thread.start_new_thread(get_sites, (i, MAX/PARTS*i) )
except Exception as e:
    print ("Error: unable to start thread: "+ str(e))

In [20]:
len(dados)

4504

In [18]:
df = pd.DataFrame(dados, columns=['data', 'titulo', 'link', 'descricao'])

In [19]:
df.to_csv('noticias_if.csv', index=False)